In [2]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

alpha=2.5
v=0.03
l=10
F0=1
n=100
sigma=0.01


def phi(w):
    if w <= 0:
      return 0
    else:
      return (F0-sigma)/(1.0 + w/(1.0-sigma))

num_moving_blocks=0
def movement(r,t,movingpoints,tpoints):

    num_moving_blocks = 0
    omega_i = r[n:]

    if (omega_i[0] > -v  ):
      num_moving_blocks += 1

    for i in range(1,n-1):
      if (omega_i[i] > -v  ):
        num_moving_blocks += 1

    if (omega_i[n-1] > -v ):
      num_moving_blocks += 1

    movementpoints.append(num_moving_blocks)
    return num_moving_blocks

def f(r,t):

    U_i=r[:n]
    omega_i=r[n:]

    fU_i = np.empty(n,float)
    fomega_i = np.empty(n,float)

    fU_i[0] = omega_i[0]
    s0 = (l**2)*(U_i[1]-U_i[0]) - U_i[0]
    if (omega_i[0] <= -v and s0 <= F0):
      omega_i[0] = -v
      fomega_i[0] = 0
    else:
      fomega_i[0] = s0 - phi(2*alpha*v + 2*alpha*omega_i[0])

    for j in range(1,n-1):
      fU_i[j] = omega_i[j]
      s = (l**2)*(U_i[j+1]-2*U_i[j]+U_i[j-1]) - U_i[j]
      if (omega_i[j] <= -v and s <= F0):
        omega_i[j] = -v
        fomega_i[j] = 0
      else:
        fomega_i[j] = s - phi(2*alpha*v + 2*alpha*omega_i[j])

    fU_i[n-1] = omega_i[n-1]
    sn = -(l**2)*(U_i[n-1]-U_i[n-2]) - U_i[n-1]
    if (omega_i[n-1] <= -v and  sn <= F0) :
      omega_i[n-1] = -v
      fomega_i[n-1] = 0
    else:
      fomega_i[n-1] = sn - phi(2*alpha*v + 2*alpha*omega_i[n-1])

    return np.concatenate([fU_i,fomega_i])

h = 0.01
tpoints = []
movementpoints = []

PV = [] #Crea una lista de listas donde se almacenarán los valores de las posiciones y velocidades.
for i in range(2*n):
    PV.append([])

vi=[]
for i in range(n):
  vi.append(-v)

pi=np.array([0.14639868, 0.14642823, 0.14536992, 0.14406593, 0.14256696,
       0.14057209, 0.13795157, 0.13592632, 0.13227203, 0.12730571,
       0.13270484, 0.13898592, 0.1445328 , 0.15089376, 0.15726128,
       0.16326989, 0.16874099, 0.17528913, 0.18097302, 0.18698079,
       0.19320957, 0.19968332, 0.20631689, 0.21187327, 0.21795447,
       0.22477593, 0.23099992, 0.23750454, 0.24302708, 0.25013794,
       0.25657937, 0.26282728, 0.26911268, 0.27567021, 0.28204161,
       0.2881219 , 0.29432719, 0.29980929, 0.30521353, 0.31067118,
       0.31517847, 0.3187704 , 0.32161814, 0.32217071, 0.32102074,
       0.32061335, 0.3283191 , 0.32766461, 0.32914185, 0.32026687,
       0.32058737, 0.32204931, 0.32141648, 0.31797471, 0.3147743 ,
       0.30954407, 0.30469395, 0.29935369, 0.29320875, 0.28719626,
       0.28123129, 0.27478833, 0.26888273, 0.26270405, 0.25617247,
       0.24952297, 0.24271201, 0.23706123, 0.23078447, 0.22365522,
       0.21758708, 0.21184845, 0.20566902, 0.19904386, 0.19352553,
       0.18685935, 0.18046327, 0.17441425, 0.16855968, 0.16248514,
       0.15654352, 0.15021671, 0.14443245, 0.13856972, 0.13289546,
       0.12687536, 0.12055148, 0.11500219, 0.10824679, 0.10252771,
       0.0964228 , 0.09064041, 0.08414705, 0.07800967, 0.07170847,
       0.07527902, 0.07782106, 0.0792699 , 0.08055309, 0.08096415])
r=np.concatenate([pi,vi])


c=-1
aux=c

K=[]
Mag0=[]
tiempo=[]

for k in range(1,10): ################### Número de eventos

  aux1=aux+1
  i=c+1
  tiempoaux=[]

  while num_moving_blocks <= n:
    t = i*h
    for j in range(2*n):
      PV[j].append(r[j])


    k1 = h*f(r,t)
    k2 = h*f(r+0.5*k1, t+0.5*h)
    k3 = h*f(r+0.5*k2, t+0.5*h)
    k4 = h*f(r+k3, t+h)
    r += (k1+2*k2+2*k3+k4)/6

    tpoints.append(t)
    tiempoaux.append(t)
    num_moving_blocks = movement(r,t,movementpoints,tpoints)
    #print(num_moving_blocks)
    if (num_moving_blocks >= 1  or t == 1000):
      break
    i+=1

  c=i+1
  while num_moving_blocks >= 0:
    t = c*h
    for j in range(2*n):
      PV[j].append(r[j])


    k1 = h*f(r,t)
    k2 = h*f(r+0.5*k1, t+0.5*h)
    k3 = h*f(r+0.5*k2, t+0.5*h)
    k4 = h*f(r+k3, t+h)
    r += (k1+2*k2+2*k3+k4)/6

    tpoints.append(t)
    tiempoaux.append(t)
    num_moving_blocks = movement(r,t,movementpoints,tpoints)
    #print(num_moving_blocks)
    if (num_moving_blocks < 1 or t == 1000 ):
      break
    c+=1

  aux=c
  D=[]
  for i in range(n):
    Di=[]
    for j in range(aux1,len(tpoints)):
      Di.append(PV[i][j]+v*tpoints[j])
    D.append(Di)

  M=[]
  for j in range(len(tpoints)-aux1):
      suma=sum([fila[j] for fila in D])
      M.append(suma)

  mag0=M[-1]-M[0]

  k
  print(k)
  K.append(k)
  Mag0.append(mag0)
  tiempo.append(t)

datos=list(zip(K, Mag0, tiempo ))

df=pd.DataFrame(datos)

1
2
3
4
5
6
7
8
9
